In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import statsmodels as stats
from scipy import stats
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from xgboost import XGBRegressor
from xgboost import plot_importance
import sklearn
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

import statsmodels as stats
from scipy import stats
import pandas as pd
import os

from sklearn.model_selection import train_test_split
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
# y = Y_OBP
import numpy as np
import re 
import matplotlib.pyplot as plt
import statsmodels as stats
from scipy import stats
import pandas as pd
import os
import numpy as np
import re 
import matplotlib.pyplot as plt
from sklearn.svm import SVR
import sklearn
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor, plot_importance
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from itertools import combinations
from tqdm import tqdm
from sklearn.metrics import mean_squared_error 
from sklearn.linear_model import LinearRegression

from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error

from lightgbm import LGBMRegressor, plot_importance
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import re
from sklearn.svm import SVR

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/빅콘테스트/data/PA70이상.csv',encoding='utf-8',index_col=0)

In [171]:
df_train

,PCODE,GYEAR,GAMENUM,T_ID,POSITION,AGE_VA,MONEY,PA_1,AB_1,R_1,H_1,2B_1,3B_1,HR_1,RBI_1,SB_1,CS_1,BB_1,HBP_1,SO_1,GDP_1,AVG_1,BARREL_1,BARREL_SUM_1,OBP_1,SLG_1,OPS_1,PA_2,AB_2,R_2,H_2,2B_2,3B_2,HR_2,RBI_2,SB_2,CS_2,BB_2,HBP_2,SO_2,GDP_2,AVG_2,BARREL_2,BARREL_SUM_2,OBP_2,SLG_2,OPS_2,PA_3,AB_3,R_3,H_3,2B_3,3B_3,HR_3,RBI_3,SB_3,CS_3,BB_3,HBP_3,SO_3,GDP_3,AVG_3,BARREL_3,BARREL_SUM_3,OBP_3,SLG_3,OPS_3,Y_SLG
0,60100,2018,70.0,0.0,1.0,30.0,5000.0,14,12,3,3,2,0,0,3,0,0,1,1,4,1,0.250000,0.375000,3,0.357143,0.416667,0.773810,8,7,1,1,1,0,0,1,0,0,1,0,2,0,0.142857,0.000000,0,0.250000,0.285714,0.535714,78,71,8,20,8,1,1,4,0,0,5,2,16,1,0.112676,0.186047,8,0.346154,0.464789,0.810943,0.533333
1,60558,2018,128.0,2.0,0.0,26.0,8300.0,67,60,10,18,3,0,4,9,3,2,5,2,14,0,0.166667,0.153846,6,0.373134,0.550000,0.923134,65,63,13,17,5,1,1,4,2,0,1,1,17,1,0.206349,0.119048,5,0.292308,0.428571,0.720879,77,67,6,17,2,0,3,5,3,3,8,0,22,3,0.089552,0.128205,5,0.333333,0.417910,0.751244,0.318182
2,60558,2019,123.0,2.0,0.0,27.0,8000.0,37,32,2,8,1,0,1,3,2,1,3,2,9,1,0.062500,0.157895,3,0.351351,0.375000,0.726351,110,91,17,22,4,0,1,7,4,1,14,2,18,1,0.186813,0.196721,12,0.355140,0.318681,0.673822,49,47,13,11,2,0,0,6,3,0,1,0,9,1,0.276596,0.031250,1,0.244898,0.276596,0.521494,0.416667
3,61102,2018,132.0,3.0,3.0,25.0,18500.0,85,75,17,26,7,0,7,18,0,0,10,0,20,4,0.226667,0.215686,11,0.423529,0.720000,1.143529,80,76,4,13,2,0,0,4,0,0,3,1,14,3,0.052632,0.071429,3,0.212500,0.197368,0.409868,73,66,7,18,5,0,2,8,0,0,4,3,12,4,0.106061,0.153846,6,0.342466,0.439394,0.781860,0.494382
4,61102,2019,132.0,3.0,3.0,26.0,21000.0,85,74,13,20,5,0,6,14,0,0,5,4,10,6,0.175676,0.211538,11,0.341176,0.581081,0.922258,90,84,7,22,4,0,0,5,0,0,3,2,22,2,0.083333,0.260000,13,0.300000,0.309524,0.609524,49,44,4,12,3,0,1,6,0,0,2,1,5,2,0.090909,0.200000,6,0.306122,0.409091,0.715213,0.543860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,69901,2019,55.0,5.0,1.0,30.0,250000.0,54,46,11,15,6,0,1,8,1,0,4,3,5,1,0.239130,0.115385,3,0.407407,0.521739,0.929147,100,90,12,17,3,1,3,16,2,1,8,2,14,4,0.133333,0.088235,6,0.270000,0.344444,0.614444,100,90,12,17,3,1,3,16,2,1,8,2,14,4,0.133333,0.088235,6,0.270000,0.344444,0.614444,0.507246
215,65898,2020,73.0,9.0,0.0,24.0,3700.0,75,68,8,21,5,0,3,12,0,0,1,0,19,2,0.117647,0.125000,6,0.309859,0.514706,0.824565,11,10,3,3,0,0,0,1,0,0,1,0,1,0,0.300000,0.125000,1,0.363636,0.300000,0.663636,14,12,3,3,1,0,1,4,0,0,1,0,5,0,0.250000,0.125000,1,0.285714,0.583333,0.869048,0.436364
216,50165,2020,117.0,3.0,0.0,25.0,300000.0,100,89,20,24,1,1,10,18,1,0,11,0,31,2,0.224719,0.277778,15,0.350000,0.640449,0.990449,91,80,16,30,5,0,10,21,0,0,10,1,18,0,0.200000,0.464286,26,0.450549,0.812500,1.263049,87,74,5,21,3,0,3,12,0,0,10,0,24,0,0.067568,0.319149,15,0.356322,0.445946,0.802268,0.545455
217,50506,2020,144.0,4.0,0.0,28.0,300000.0,91,81,14,29,10,0,3,9,3,0,7,3,7,3,0.172840,0.171875,11,0.428571,0.592593,1.021164,93,81,9,17,4,0,4,17,2,0,7,2,15,2,0.111111,0.155172,9,0.282609,0.407407,0.690016,95,86,14,29,5,1,0,12,4,0,7,0,11,2,0.162791,0.128571,9,0.378947,0.418605,0.797552,0.329193


In [ ]:
df_train = df_train[df_train['Y_SLG']>=0.3]
df_train = df_train.drop('Y_OBP',axis=1)
df_train

,PCODE,GYEAR,GAMENUM,T_ID,POSITION,AGE_VA,MONEY,PA_1,AB_1,R_1,H_1,2B_1,3B_1,HR_1,RBI_1,SB_1,CS_1,BB_1,HBP_1,SO_1,GDP_1,AVG_1,BARREL_1,BARREL_SUM_1,OBP_1,SLG_1,OPS_1,PA_2,AB_2,R_2,H_2,2B_2,3B_2,HR_2,RBI_2,SB_2,CS_2,BB_2,HBP_2,SO_2,GDP_2,AVG_2,BARREL_2,BARREL_SUM_2,OBP_2,SLG_2,OPS_2,PA_3,AB_3,R_3,H_3,2B_3,3B_3,HR_3,RBI_3,SB_3,CS_3,BB_3,HBP_3,SO_3,GDP_3,AVG_3,BARREL_3,BARREL_SUM_3,OBP_3,SLG_3,OPS_3,Y_SLG
0,60100,2018,70.0,0.0,1.0,30.0,5000.0,14,12,3,3,2,0,0,3,0,0,1,1,4,1,0.250000,0.375000,3,0.357143,0.416667,0.773810,8,7,1,1,1,0,0,1,0,0,1,0,2,0,0.142857,0.000000,0,0.250000,0.285714,0.535714,78,71,8,20,8,1,1,4,0,0,5,2,16,1,0.112676,0.186047,8,0.346154,0.464789,0.810943,0.533333
4,60558,2018,128.0,2.0,0.0,26.0,8300.0,67,60,10,18,3,0,4,9,3,2,5,2,14,0,0.166667,0.153846,6,0.373134,0.550000,0.923134,65,63,13,17,5,1,1,4,2,0,1,1,17,1,0.206349,0.119048,5,0.292308,0.428571,0.720879,77,67,6,17,2,0,3,5,3,3,8,0,22,3,0.089552,0.128205,5,0.333333,0.417910,0.751244,0.318182
5,60558,2019,123.0,2.0,0.0,27.0,8000.0,37,32,2,8,1,0,1,3,2,1,3,2,9,1,0.062500,0.157895,3,0.351351,0.375000,0.726351,110,91,17,22,4,0,1,7,4,1,14,2,18,1,0.186813,0.196721,12,0.355140,0.318681,0.673822,49,47,13,11,2,0,0,6,3,0,1,0,9,1,0.276596,0.031250,1,0.244898,0.276596,0.521494,0.416667
10,61102,2018,132.0,3.0,3.0,25.0,18500.0,85,75,17,26,7,0,7,18,0,0,10,0,20,4,0.226667,0.215686,11,0.423529,0.720000,1.143529,80,76,4,13,2,0,0,4,0,0,3,1,14,3,0.052632,0.071429,3,0.212500,0.197368,0.409868,73,66,7,18,5,0,2,8,0,0,4,3,12,4,0.106061,0.153846,6,0.342466,0.439394,0.781860,0.494382
11,61102,2019,132.0,3.0,3.0,26.0,21000.0,85,74,13,20,5,0,6,14,0,0,5,4,10,6,0.175676,0.211538,11,0.341176,0.581081,0.922258,90,84,7,22,4,0,0,5,0,0,3,2,22,2,0.083333,0.260000,13,0.300000,0.309524,0.609524,49,44,4,12,3,0,1,6,0,0,2,1,5,2,0.090909,0.200000,6,0.306122,0.409091,0.715213,0.543860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,69901,2019,55.0,5.0,1.0,30.0,250000.0,54,46,11,15,6,0,1,8,1,0,4,3,5,1,0.239130,0.115385,3,0.407407,0.521739,0.929147,100,90,12,17,3,1,3,16,2,1,8,2,14,4,0.133333,0.088235,6,0.270000,0.344444,0.614444,100,90,12,17,3,1,3,16,2,1,8,2,14,4,0.133333,0.088235,6,0.270000,0.344444,0.614444,0.507246
516,65898,2020,73.0,9.0,0.0,24.0,3700.0,75,68,8,21,5,0,3,12,0,0,1,0,19,2,0.117647,0.125000,6,0.309859,0.514706,0.824565,11,10,3,3,0,0,0,1,0,0,1,0,1,0,0.300000,0.125000,1,0.363636,0.300000,0.663636,14,12,3,3,1,0,1,4,0,0,1,0,5,0,0.250000,0.125000,1,0.285714,0.583333,0.869048,0.436364
521,50165,2020,117.0,3.0,0.0,25.0,300000.0,100,89,20,24,1,1,10,18,1,0,11,0,31,2,0.224719,0.277778,15,0.350000,0.640449,0.990449,91,80,16,30,5,0,10,21,0,0,10,1,18,0,0.200000,0.464286,26,0.450549,0.812500,1.263049,87,74,5,21,3,0,3,12,0,0,10,0,24,0,0.067568,0.319149,15,0.356322,0.445946,0.802268,0.545455
524,50506,2020,144.0,4.0,0.0,28.0,300000.0,91,81,14,29,10,0,3,9,3,0,7,3,7,3,0.172840,0.171875,11,0.428571,0.592593,1.021164,93,81,9,17,4,0,4,17,2,0,7,2,15,2,0.111111,0.155172,9,0.282609,0.407407,0.690016,95,86,14,29,5,1,0,12,4,0,7,0,11,2,0.162791,0.128571,9,0.378947,0.418605,0.797552,0.329193


In [ ]:
df_train.reset_index(drop=True, inplace=True)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/빅콘테스트/data/2021_x_test.csv',encoding='utf-8')

In [ ]:
df

,PCODE,GYEAR,GAMENUM,T_ID,POSITION,AGE_VA,MONEY,PA_1,AB_1,R_1,H_1,2B_1,3B_1,HR_1,RBI_1,SB_1,CS_1,BB_1,HBP_1,SO_1,GDP_1,AVG_1,BARREL_1,BARREL_SUM_1,OBP_1,SLG_1,OPS_1,PA_2,AB_2,R_2,H_2,2B_2,3B_2,HR_2,RBI_2,SB_2,CS_2,BB_2,HBP_2,SO_2,GDP_2,AVG_2,BARREL_2,BARREL_SUM_2,OBP_2,SLG_2,OPS_2,PA_3,AB_3,R_3,H_3,2B_3,3B_3,HR_3,RBI_3,SB_3,CS_3,BB_3,HBP_3,SO_3,GDP_3,AVG_3,BARREL_3,BARREL_SUM_3,OBP_3,SLG_3,OPS_3
0,67341,2021,79.0,9.0,1.0,22.0,55000.0,110,93,15,25,5,1,0,12,3,2,15,1,9,3,0.161290,0.132353,9,0.372727,0.344086,0.716813,99,82,21,37,13,2,1,21,2,0,12,3,5,2,0.256098,0.191781,14,0.525253,0.695122,1.220374,117,98,15,31,11,1,2,13,1,1,17,0,7,2,0.153061,0.141176,12,0.410256,0.510204,0.920460
1,67872,2021,78.0,7.0,0.0,35.0,900000.0,90,77,13,20,3,0,5,16,0,0,11,1,17,1,0.168831,0.163636,9,0.355556,0.493506,0.849062,96,78,14,20,2,0,4,11,0,0,17,0,17,3,0.179487,0.203390,12,0.385417,0.435897,0.821314,108,90,17,18,2,0,8,16,1,0,17,0,19,3,0.188889,0.131148,8,0.324074,0.488889,0.812963
2,68050,2021,75.0,2.0,0.0,21.0,31000.0,106,91,10,37,4,1,2,21,2,0,12,0,10,3,0.109890,0.202899,14,0.462264,0.538462,1.000726,97,79,12,33,6,0,4,23,1,2,18,0,14,1,0.151899,0.129630,7,0.525773,0.645570,1.171343,97,77,19,29,5,0,3,13,4,0,20,0,14,0,0.246753,0.288462,15,0.505155,0.558442,1.063596
3,75847,2021,74.0,7.0,0.0,34.0,120000.0,92,74,12,19,1,0,5,12,2,2,13,2,23,0,0.162162,0.150943,8,0.369565,0.472973,0.842538,91,70,21,25,6,0,7,22,3,0,16,4,17,0,0.300000,0.351852,19,0.494505,0.742857,1.237363,103,83,17,25,4,0,8,21,2,0,13,6,11,3,0.204819,0.227273,15,0.427184,0.638554,1.065739
4,76232,2021,73.0,5.0,3.0,34.0,150000.0,92,77,14,26,6,1,4,23,1,0,10,4,13,1,0.181818,0.166667,10,0.434783,0.597403,1.032185,89,71,19,25,4,0,7,21,0,0,17,1,4,1,0.267606,0.196721,12,0.483146,0.704225,1.187371,101,85,18,31,6,0,8,24,1,1,14,1,11,2,0.211765,0.263889,19,0.455446,0.717647,1.173093
5,76290,2021,73.0,3.0,1.0,33.0,100000.0,98,82,7,24,3,0,4,18,0,0,14,0,12,1,0.085366,0.234375,15,0.387755,0.475610,0.863365,102,84,19,29,5,0,4,14,0,0,14,1,8,1,0.226190,0.238095,15,0.431373,0.547619,0.978992,98,86,14,24,4,0,4,14,0,0,12,0,5,2,0.162791,0.253731,17,0.367347,0.465116,0.832463
6,78224,2021,68.0,6.0,1.0,32.0,76000.0,93,74,18,21,2,1,7,26,0,0,18,1,19,0,0.243243,0.254902,13,0.430108,0.621622,1.051729,95,81,13,24,2,0,4,16,0,0,13,0,18,1,0.160494,0.267857,15,0.389474,0.469136,0.858609,91,73,12,15,1,0,4,11,0,0,15,2,25,2,0.164384,0.292683,12,0.351648,0.383562,0.735210
7,78513,2021,77.0,4.0,0.0,35.0,50000.0,107,87,13,27,5,0,1,14,1,1,17,2,7,3,0.149425,0.176471,12,0.433962,0.402299,0.836261,94,81,12,27,6,0,1,12,1,0,9,1,13,1,0.148148,0.206897,12,0.393617,0.444444,0.838061,115,98,20,33,8,0,2,19,1,0,13,2,11,1,0.204082,0.202532,16,0.417391,0.479592,0.896983
8,79192,2021,61.0,3.0,1.0,31.0,30000.0,48,45,3,13,1,0,1,1,0,0,3,0,7,0,0.066667,0.205882,7,0.333333,0.377778,0.711111,94,87,18,26,8,0,5,24,2,1,5,2,21,3,0.206897,0.233333,14,0.351064,0.563218,0.914282,98,80,18,28,2,0,6,25,1,0,11,4,14,1,0.225000,0.292308,19,0.438776,0.600000,1.038776
9,79215,2021,59.0,6.0,1.0,30.0,48000.0,94,79,15,30,5,0,2,16,1,0,11,2,10,3,0.189873,0.218750,14,0.457447,0.518987,0.976434,75,68,11,19,4,0,0,9,2,0,6,1,10,4,0.161765,0.177778,8,0.346667,0.338235,0.684902,54,48,7,16,4,0,0,7,3,0,5,0,7,3,0.145833,0.078947,3,0.388889,0.416667,0.805556


In [ ]:
X_train = df_train.drop('Y_SLG',axis=1)
y_train = df_train['Y_SLG']
X_test = df

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold

def get_stacking_data(model, X_train, y_train, X_test, n_folds=5):
    kfold = KFold(n_splits = n_folds, shuffle=True)
    
    # 최종 모델에서 사용할 데이터 셋 셋팅(0 값으로)
    # 만약 shape가 (100, 10) 이었으면 폴드의 검증 과정에서 저장할 데이터는 (100, 1) 모양을 갖게 한다.
    train_fold_predict = np.zeros((X_train.shape[0], 1))
    # test는 X_test 값을 이용해서 매 폴드마다 예측을 하기 때문에 (100, fold개수) 만큼의 shape를 갖게 된다.
    # 그래서 해당 폴드마다 X_test의 예측 값을 해당 fold에 해당되는 열에 넣는다.
    test_predict = np.zeros((X_test.shape[0], n_folds))
    print("model : ", model.__class__.__name__)
    
    for cnt, (train_index, valid_index) in enumerate(kfold.split(X_train)):
        X_train_ = X_train.iloc[train_index]
        y_train_ = y_train.iloc[train_index]
        X_validation = X_train.iloc[valid_index]
        
        #학습
        model.fit(X_train_, y_train_)
        
        #해당 폴드에서 학습된 모델에다가 검증 데이터(X_validation)로 예측 후 저장
        train_fold_predict[valid_index, :] = model.predict(X_validation).reshape(-1, 1)
        
        #해당 폴드에서 생성된 모델에게 원본 테스트 데이터(X_test)를 이용해서 예측을 수행하고 저장
        test_predict[:, cnt] = model.predict(X_test)
    
    #for문이 끝나면 test_pred는 평균을 내서 하나로 합친다.
    test_predict_mean = np.mean(test_predict, axis =1).reshape(-1, 1)
    
    return train_fold_predict, test_predict_mean

In [ ]:
model1 = Lasso(alpha=1000) # 0.05
model2 = Ridge(alpha=500000) # 10000
model3 = ElasticNet(alpha= 100000, l1_ratio=0.01)
model4 = LGBMRegressor(lambda_l1= 0, lamda_l2=0, learning_rate=0.1, max_depth=-1, n_estimaotrs= 5000, num_leaves= 7)

In [ ]:
model1_train, model1_test = get_stacking_data(model1, X_train, y_train, X_test)
model2_train, model2_test = get_stacking_data(model2, X_train, y_train, X_test)
model3_train, model3_test = get_stacking_data(model3, X_train, y_train, X_test)

model :  Lasso
model :  Ridge
model :  ElasticNet


In [ ]:
new_X_train = np.concatenate((model1_train, model2_train, model3_train), axis = 1)
new_X_test = np.concatenate((model1_test, model2_test,model3_test), axis = 1)

In [ ]:
print("원본 : ", X_train.shape, X_test.shape)
print("새로운 : ", new_X_train.shape, new_X_test.shape)

원본 :  (157, 67) (10, 67)
새로운 :  (157, 3) (10, 3)


In [ ]:
model4.fit(new_X_train, y_train)
stack_pred = model4.predict(new_X_test)

In [ ]:
stack_pred

array([0.33319093, 0.47938009, 0.34838976, 0.48908435, 0.54155456,
       0.49646934, 0.53185804, 0.42049458, 0.45576647, 0.42932701])

In [ ]:
model1_train, model1_test = get_stacking_data(model1, X_train, y_train, X_test)
model2_train, model2_test = get_stacking_data(model2, X_train, y_train, X_test)
model3_train, model3_test = get_stacking_data(model3, X_train, y_train, X_test)
new_X_train = np.concatenate((model1_train, model2_train, model3_train), axis = 1)
new_X_test = np.concatenate((model1_test, model2_test,model3_test), axis = 1)
model4.fit(new_X_train, y_train)
stack_pred2 = model4.predict(new_X_test)
print(stack_pred2)
print(stack_pred2.mean())

model :  Lasso
model :  Ridge
model :  ElasticNet
[0.44003373 0.4953823  0.40906047 0.46485775 0.46028406 0.51937261
 0.52304757 0.51115747 0.46841911 0.47556797]
0.47671830395413173


In [ ]:
model1_train, model1_test = get_stacking_data(model1, X_train, y_train, X_test)
model2_train, model2_test = get_stacking_data(model2, X_train, y_train, X_test)
model3_train, model3_test = get_stacking_data(model3, X_train, y_train, X_test)
new_X_train = np.concatenate((model1_train, model2_train, model3_train), axis = 1)
new_X_test = np.concatenate((model1_test, model2_test,model3_test), axis = 1)
model4.fit(new_X_train, y_train)
stack_pred3 = model4.predict(new_X_test)
print(stack_pred3.mean())
stack_pred3

model :  Lasso
model :  Ridge
model :  ElasticNet
0.45283843346400215


array([0.42251744, 0.48751107, 0.42222752, 0.39979285, 0.58511698,
       0.39206134, 0.4164477 , 0.46066608, 0.45453307, 0.4875103 ])

In [ ]:
model1_train, model1_test = get_stacking_data(model1, X_train, y_train, X_test)
model2_train, model2_test = get_stacking_data(model2, X_train, y_train, X_test)
model3_train, model3_test = get_stacking_data(model3, X_train, y_train, X_test)
new_X_train = np.concatenate((model1_train, model2_train, model3_train), axis = 1)
new_X_test = np.concatenate((model1_test, model2_test,model3_test), axis = 1)
model4.fit(new_X_train, y_train)
stack_pred4 = model4.predict(new_X_test)
print(stack_pred4.mean())
stack_pred4

model :  Lasso
model :  Ridge
model :  ElasticNet
0.48236865284246805


array([0.45362464, 0.49840529, 0.38453059, 0.52157908, 0.47916378,
       0.54219527, 0.54053415, 0.50723085, 0.40341777, 0.4930051 ])

In [ ]:
model1_train, model1_test = get_stacking_data(model1, X_train, y_train, X_test)
model2_train, model2_test = get_stacking_data(model2, X_train, y_train, X_test)
model3_train, model3_test = get_stacking_data(model3, X_train, y_train, X_test)
new_X_train = np.concatenate((model1_train, model2_train, model3_train), axis = 1)
new_X_test = np.concatenate((model1_test, model2_test,model3_test), axis = 1)
model4.fit(new_X_train, y_train)
stack_pred5 = model4.predict(new_X_test)
print(stack_pred5.mean())
stack_pred5

model :  Lasso
model :  Ridge
model :  ElasticNet
0.4582921428237004


array([0.44874958, 0.49469372, 0.43058346, 0.50695776, 0.43849803,
       0.48180671, 0.41428616, 0.46647394, 0.46070981, 0.44016226])

In [ ]:
model1_train, model1_test = get_stacking_data(model1, X_train, y_train, X_test)
model2_train, model2_test = get_stacking_data(model2, X_train, y_train, X_test)
model3_train, model3_test = get_stacking_data(model3, X_train, y_train, X_test)
new_X_train = np.concatenate((model1_train, model2_train, model3_train), axis = 1)
new_X_test = np.concatenate((model1_test, model2_test,model3_test), axis = 1)
model4.fit(new_X_train, y_train)
stack_pred6 = model4.predict(new_X_test)
print(stack_pred6.mean())
stack_pred6

model :  Lasso
model :  Ridge
model :  ElasticNet
0.42127320295985626


array([0.39089729, 0.48870592, 0.36704413, 0.43673307, 0.48400545,
       0.42214788, 0.44920837, 0.41327009, 0.39838815, 0.36233168])

In [ ]:
model1_train, model1_test = get_stacking_data(model1, X_train, y_train, X_test)
model2_train, model2_test = get_stacking_data(model2, X_train, y_train, X_test)
model3_train, model3_test = get_stacking_data(model3, X_train, y_train, X_test)
new_X_train = np.concatenate((model1_train, model2_train, model3_train), axis = 1)
new_X_test = np.concatenate((model1_test, model2_test,model3_test), axis = 1)
model4.fit(new_X_train, y_train)
stack_pred7 = model4.predict(new_X_test)
print(stack_pred7.mean())
stack_pred7

model :  Lasso
model :  Ridge
model :  ElasticNet
0.4590304015215441


array([0.39699026, 0.48770648, 0.37016834, 0.47395486, 0.4788417 ,
       0.45930792, 0.47904438, 0.49177525, 0.46275418, 0.48976065])

In [ ]:
model1_train, model1_test = get_stacking_data(model1, X_train, y_train, X_test)
model2_train, model2_test = get_stacking_data(model2, X_train, y_train, X_test)
model3_train, model3_test = get_stacking_data(model3, X_train, y_train, X_test)
new_X_train = np.concatenate((model1_train, model2_train, model3_train), axis = 1)
new_X_test = np.concatenate((model1_test, model2_test,model3_test), axis = 1)
model4.fit(new_X_train, y_train)
stack_pred8 = model4.predict(new_X_test)
print(stack_pred8.mean())
stack_pred8

model :  Lasso
model :  Ridge
model :  ElasticNet
0.47106335008653755


array([0.43033228, 0.48397098, 0.47366946, 0.45163078, 0.43124559,
       0.54262932, 0.4527758 , 0.4364263 , 0.49173047, 0.51622253])

In [ ]:
model1_train, model1_test = get_stacking_data(model1, X_train, y_train, X_test)
model2_train, model2_test = get_stacking_data(model2, X_train, y_train, X_test)
model3_train, model3_test = get_stacking_data(model3, X_train, y_train, X_test)
new_X_train = np.concatenate((model1_train, model2_train, model3_train), axis = 1)
new_X_test = np.concatenate((model1_test, model2_test,model3_test), axis = 1)
model4.fit(new_X_train, y_train)
stack_pred9 = model4.predict(new_X_test)
print(stack_pred9.mean())
stack_pred9

model :  Lasso
model :  Ridge
model :  ElasticNet
0.49594521920039414


array([0.51848878, 0.48318901, 0.51846375, 0.48161938, 0.55185893,
       0.41706784, 0.49283725, 0.51596264, 0.46966269, 0.51030192])

In [ ]:
model1_train, model1_test = get_stacking_data(model1, X_train, y_train, X_test)
model2_train, model2_test = get_stacking_data(model2, X_train, y_train, X_test)
model3_train, model3_test = get_stacking_data(model3, X_train, y_train, X_test)
new_X_train = np.concatenate((model1_train, model2_train, model3_train), axis = 1)
new_X_test = np.concatenate((model1_test, model2_test,model3_test), axis = 1)
model4.fit(new_X_train, y_train)
stack_pred10 = model4.predict(new_X_test)
print(stack_pred10.mean())
stack_pred10

model :  Lasso
model :  Ridge
model :  ElasticNet
0.4688408532437286


array([0.46113573, 0.48587023, 0.41195304, 0.51082934, 0.51481856,
       0.49924522, 0.46046575, 0.44449009, 0.47241208, 0.42718851])